In [1]:
import os
# 【硬性要求】加载环境变量 (确保可以调用 OpenAI API)
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
# 导入自定义工具所需的装饰器
from langchain_core.tools import tool

# ==========================================
# 1. 定义自定义工具 (Defining Custom Tool)
# ==========================================
# 【核心魔法】：使用 @tool 装饰器。
# 注意必须写清晰的类型注解 (str) 和文档字符串 (""")，因为这是暴露给 LangChain 框架的元数据！
@tool
def get_flight_status(flight: str) -> str:
    """获取指定航班的实时航班状态、出发时间、到达时间和登机口。"""
    # 在真实科研/工程场景中，这里会调用 Requests 去请求航空公司的真实 API。
    # 这里我们使用老师讲稿中的硬编码静态字符串
    return f"Flight {flight} status: On-time. Departure time: 5:20 PM. Arrival time: Expected 8:10 PM. Gate: B12."

print("=== 🛠️ 探查自定义工具的自动生成架构 ===")
print(f"工具名称: {get_flight_status.name}")
print(f"工具描述: {get_flight_status.description}")
print(f"参数架构: {get_flight_status.args}\n") # 你会看到自动解析出了 flight 变量

# ==========================================
# 2. 手动执行工具并搭建 RAG 风格管道 (Execution & LCEL Setup)
# ==========================================
# 设定查询的航班号，老师最喜欢的阿联酋航空 EK524
flight_no = "EK524"

# 步骤 A: 提前运行工具获取实时上下文 (模拟 RAG 的检索步骤)
# 【现代写法】: 旧版本是 GetFlightStatus.run()，现在标准写法是 .invoke() 并传入字典
real_time_context = get_flight_status.invoke({"flight": flight_no})
print(f"📡 实时获取到的上下文: {real_time_context}\n")

# 步骤 B: 搭建带有严格护栏的 Prompt
prompt_text = """
Answer the query: {query} about flight: {flight} in one word or a short phrase. 
Answer only if you find the details in the context.

Context: {context}
"""
prompt = PromptTemplate.from_template(prompt_text)

# 【规范更新】旧版：llm=OpenAI()。现代写法：ChatOpenAI()，并设置 temperature=0 保证事实性。
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
output_parser = StrOutputParser()

# 组装链条
chain = prompt | llm | output_parser

# ==========================================
# 3. 动态提问测试 (Testing various queries)
# ==========================================
if __name__ == "__main__":
    queries = ["status", "departure time", "arrival time", "gate"]
    print("=== ✈️ 航班状态查询系统测试 ===")
    
    for q in queries:
        # 将实时抓取到的 context 和其他变量一起丢入链条执行
        ans = chain.invoke({
            "context": real_time_context, 
            "query": q, 
            "flight": flight_no
        })
        print(f"❓ 询问 '{q}' -> 🤖 AI 提取回答: {ans.strip()}")

=== 🛠️ 探查自定义工具的自动生成架构 ===
工具名称: get_flight_status
工具描述: 获取指定航班的实时航班状态、出发时间、到达时间和登机口。
参数架构: {'flight': {'title': 'Flight', 'type': 'string'}}

📡 实时获取到的上下文: Flight EK524 status: On-time. Departure time: 5:20 PM. Arrival time: Expected 8:10 PM. Gate: B12.

=== ✈️ 航班状态查询系统测试 ===
❓ 询问 'status' -> 🤖 AI 提取回答: On-time
❓ 询问 'departure time' -> 🤖 AI 提取回答: 5:20 PM
❓ 询问 'arrival time' -> 🤖 AI 提取回答: Expected 8:10 PM
❓ 询问 'gate' -> 🤖 AI 提取回答: Gate B12
